In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import sys
sys.path.insert(0, "python")
from network import *
from protocols import *
from adversary import *
from simulator import *

/home/fberes/.conda/envs/ethsim/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
a = [1,2,3]
a.pop(0)

1

In [10]:
G.copy()

In [3]:
G = nx.Graph()
G.add_weighted_edges_from([(1,2,0.3),(1,3,0.4),(3,4,0.1)], weight="latency")
G.edges(data=True)

EdgeDataView([(1, 2, {'latency': 0.3}), (1, 3, {'latency': 0.4}), (3, 4, {'latency': 0.1})])

In [4]:
net = Network(graph=G, edge_weight="custom")
net.graph.edges(data=True)

{(1, 2): 0.3, (1, 3): 0.4, (3, 4): 0.1}


EdgeDataView([(1, 2, {'latency': 0.3}), (1, 3, {'latency': 0.4}), (3, 4, {'latency': 0.1})])

In [5]:
net.edge_weights

{(1, 2): 0.3, (1, 3): 0.4, (3, 4): 0.1}

# Priority queue

In [ ]:
from queue import PriorityQueue
pq = PriorityQueue(10)

In [ ]:
pq.put((2,"a"))

In [ ]:
pq.put((3,"b"))

In [ ]:
pq.put((1,"c"))

In [ ]:
pq.queue

In [ ]:
pq.put((1,"b"))

In [ ]:
pq.get()

# Dandelion example

In [ ]:
net = Network(400, 40, edge_weight="unweighted")

In [ ]:
dp = DandelionProtocol(net, 0.1)

In [ ]:
nx.draw(dp.line_graph, node_size=20)

In [ ]:
adv = Adversary(net, 0.1)
sim = Simulator(dp, adv, 10, verbose=False)
sim.run(coverage_threshold=0.95)

In [ ]:
%%time
evaluator = Evaluator(sim, "first_reach")
print(evaluator.get_report())

In [ ]:
%%time
evaluator = Evaluator(sim, "shortest_path")
print(evaluator.get_report())

In [ ]:
%%time
evaluator = Evaluator(sim, "dummy")
print(evaluator.get_report())

In [ ]:
alma

# Experiment example

In [ ]:
def run_and_eval(net: Network, adv: Adversary, protocol: Protocol):
    sim = Simulator(protocol, adv, int(network_size / 10), verbose=False)
    sim.run()
    evaluator = Evaluator(sim)
    report = evaluator.get_report()
    report["protocol"] = str(protocol)
    report["adversary_ratio"] = adv.ratio
    return report

In [ ]:
network_size = 1000
degree = 50
num_trials = 10
adversary_ratios = [0.01,0.025,0.05,0.075,0.1]
results = []

In [ ]:
%%time
# it could be parallelized
for adv_ratio in adversary_ratios:
    for _ in range(num_trials):
        # this part cannot be parallelized: it is important that different protocols will be evaluated with the same network and adversary setting
        net = Network(network_size, degree)
        adv = Adversary(net, adv_ratio)
        # BUT: messages won't be the same!!!
        results.append(run_and_eval(net, adv, BroadcastProtocol(net)))
        results.append(run_and_eval(net, adv, DandelionProtocol(net, 0.5)))
        results.append(run_and_eval(net, adv, DandelionProtocol(net, 0.25)))
    print(adv_ratio)

In [ ]:
import seaborn as sns

In [ ]:
results_df = pd.DataFrame(results)
results_df.head()

### Dummy adversary predicts message source for Dandelion much worse than in case of ismple broadcasting

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(15,15))
sns.boxplot(data=results_df, x="adversary_ratio", y="hit_ratio", hue="protocol", ax=ax[0][0])
sns.boxplot(data=results_df, x="adversary_ratio", y="inverse_rank", hue="protocol", ax=ax[0][1])
sns.boxplot(data=results_df, x="adversary_ratio", y="ndcg", hue="protocol", ax=ax[1][0])
sns.boxplot(data=results_df, x="adversary_ratio", y="entropy", hue="protocol", ax=ax[1][1])

### While the fraction of nodes reached by messages are the same range for all protocols

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,6))
sns.boxplot(data=results_df, x="adversary_ratio", y="message_spread_ratio", hue="protocol", ax=ax)